In [5]:
# https://github.com/tensorflow/examples/blob/master/lite/examples/object_detection/raspberry_pi/detect.py
import cv2
from tflite_support.task import core
from tflite_support.task import processor
from tflite_support.task import vision
import time

In [3]:
import os

os.chdir('../src')
print(os.getcwd())

/home/matrik/Yandex.Disk/CM3070_FP/code/src


In [4]:
in_video_path = './evaluation/DCSASS Dataset/Stealing/Stealing002_x264.mp4/Stealing002_x264_9.mp4'
store_out_video = True
output_codec="VP80"
output_ext = "webm"

cap = cv2.VideoCapture(in_video_path)
if not cap.isOpened():
    print(f"Failed to open video: {in_video_path}")
    # return set()

# Get the frame width, height, and FPS
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define the output path
output_path = None
if store_out_video and in_video_path.endswith("_unannotated.avi"):
    output_path = in_video_path.replace("_unannotated.avi", f".{output_ext}")
elif store_out_video:
    output_path = in_video_path[: in_video_path.rfind(".")] + f"_annotated.{output_ext}"

# Initialize the video writer
out = (
    cv2.VideoWriter(
        output_path, cv2.VideoWriter_fourcc(*output_codec), fps, (frame_width, frame_height)
    )
    if store_out_video
    else None
)


OpenCV: FFMPEG: tag 0x30385056/'VP80' is not supported with codec id 139 and format 'webm / WebM'


In [15]:
# Initialize the object detection model
base_options = core.BaseOptions(
    file_name='/home/matrik/Yandex.Disk/CM3070_FP/code/models/efficientdet-lite/efficientdet-lite4-detection-metadata.tflite', use_coral=False, num_threads=4)
detection_options = processor.DetectionOptions(
    max_results=3, score_threshold=0.5)
options = vision.ObjectDetectorOptions(
    base_options=base_options, detection_options=detection_options)
detector = vision.ObjectDetector.create_from_options(options)

In [ ]:
processed_frames = 0
start_time = time.time()


while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Read the frame and resize it
    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Create a TensorImage object from the RGB image.
    input_tensor = vision.TensorImage.create_from_array(frame_rgb)

    # Run object detection estimation using the model.
    detection_result = detector.detect(input_tensor)

    processed_frames += 1